In [1]:
import os, random, re, math, ast
import numpy as np
import pandas as pd
import warnings
from string import digits
warnings.filterwarnings('ignore')

# 1. Data Acquicision and Cleaning

## Source: Allmusic  
https://www.allmusic.com/

In [2]:
with open('../allmusic/data/artist.jsonl') as f:
    lines = f.readlines()
artist_df = pd.DataFrame([eval(i) for i in lines])
artist_df.biography = list(map(lambda x: x[0] if isinstance(x, list) else None, artist_df.biography))
artist_df.active_years = list(map(lambda x: x[0] if isinstance(x, list) else None, artist_df.active_years))
artist_df.born_in = list(map(lambda x: ', '.join(x) if isinstance(x, list) else None, artist_df.born_in))
artist_df.aliases = list(map(lambda x: x[0].strip() if isinstance(x, list) else None, artist_df.aliases))
artist_df.group_members = list(map(lambda x: [member.strip() for member in x] if isinstance(x, list) else None, artist_df.group_members))
artist_df.group_members_url = list(map(lambda x: ['https://www.allmusic.com' + url for url in x] if isinstance(x, list) else None,artist_df.group_members_url))
artist_df.head()

,name,url,biography,active_years,born_in,genres,styles,aliases,group_members,group_members_url
0,Glenn Tilbrook,https://www.allmusic.com/artist/glenn-tilbrook...,"The main vocalist and melodicist for Squeeze, ...",1970s - 2020s,"August 31, 1957, Dulwich, London, England",[Pop/Rock],"[Adult Alternative Pop/Rock, Alternative/Indie...",Glenn Martin Tilbrook,None,None
1,Chris Difford,https://www.allmusic.com/artist/chris-difford-...,Respected English singer/songwriter who has ma...,1970s - 2020s,"November 4, 1954, Greenwich, London, England",[Pop/Rock],"[Adult Alternative Pop/Rock, Alternative/Indie...",Christopher Henry Difford,None,None
2,Simon LeBon,https://www.allmusic.com/artist/simon-lebon-mn...,None,1970s - 2020s,"October 27, 1958, Bushey, Hertfordshire, England","[Pop/Rock, Electronic]","[Dance-Pop, Dance-Rock, New Romantic, New Wave...",None,None,None
3,Horace Silver,https://www.allmusic.com/artist/horace-silver-...,One of the leading composers and pioneers of h...,1950s - 2000s,"September 2, 1928, Norwalk, CT",[Jazz],"[Fusion, Hard Bop, Soul Jazz, Jazz Instrument,...",Horace Ward Martin Tavares Silva,None,None
4,Jon Hendricks,https://www.allmusic.com/artist/jon-hendricks-...,Vocalese singer who mastered the art of adding...,1950s - 2010s,"September 16, 1921, Newark, OH","[Vocal, Jazz]","[Bop, Vocal Jazz, Vocalese, Big Band, Hard Bop...",John Carl Hendricks,None,None


In [3]:
with open('../allmusic/data/song.jsonl') as f:
    lines = f.readlines()
allmusic_df = pd.DataFrame([eval(i) for i in lines])
allmusic_df.composer_url = list(map(lambda x: ['https://www.allmusic.com' + url for url in x] if isinstance(x, list) else None,allmusic_df.composer_url))
allmusic_df.release_year = list(map(lambda x: str(x[0]) if isinstance(x, list) else None,allmusic_df.release_year))
allmusic_df.genres = list(map(lambda x: [genre.translate(str.maketrans('', '', digits)).replace('()', '').strip() for genre in x] if isinstance(x, list) else None,allmusic_df.genres))
allmusic_df.styles = list(map(lambda x: [style.translate(str.maketrans('', '', digits)).replace('()', '').strip() for style in x] if isinstance(x, list) else None,allmusic_df.styles))


allmusic_df = allmusic_df.drop(columns = ['moods', 'themes'])
allmusic_df = allmusic_df.reset_index()
allmusic_df.rename(columns={'index':'id'}, inplace = True)
allmusic_df.id = list(map(lambda x: str(x), allmusic_df.id))
allmusic_df.head()

,id,title,url,performer,performer_url,composer,composer_url,release_year,genres,styles,in_album,album_url
0,0,Pulling Mussels (From the Shell),https://www.allmusic.com/song/pulling-mussels-...,[Squeeze],[https://www.allmusic.com/artist/squeeze-mn000...,"[Chris Difford, Glenn Tilbrook]",[https://www.allmusic.com/artist/chris-difford...,1980,[Pop/Rock],"[Power Pop, New Wave, Alternative/Indie Rock, ...","[Argybargy, Singles 45's and Under, Greatest H...",[https://www.allmusic.com/album/argybargy-mw00...
1,1,Girls on Film,https://www.allmusic.com/song/girls-on-film-mt...,[Duran Duran],[https://www.allmusic.com/artist/duran-duran-m...,"[Simon LeBon, Nick Rhodes, Andy Taylor, John T...",[https://www.allmusic.com/artist/simon-lebon-m...,1983,[Pop/Rock],"[Synth Pop, New Romantic, New Wave, Punk/New W...","[Seven and the Ragged Tiger, Duran Duran [Long...",[https://www.allmusic.com/album/seven-and-the-...
2,2,Doodlin',https://www.allmusic.com/song/doodlin-mt000465...,[Horace Silver],[https://www.allmusic.com/artist/horace-silver...,"[Jon Hendricks, Horace Silver]",[https://www.allmusic.com/artist/jon-hendricks...,1955,[Jazz],[Hard Bop],"[Horace Silver and the Jazz Messengers, Jazz E...",[https://www.allmusic.com/album/horace-silver-...
3,3,Rock Me on the Water,https://www.allmusic.com/song/rock-me-on-the-w...,[Jackson Browne],[https://www.allmusic.com/artist/jackson-brown...,[Jackson Browne],[https://www.allmusic.com/artist/jackson-brown...,1971,[Pop/Rock],"[Singer/Songwriter, Soft Rock, Contemporary Po...","[Jackson Browne, Jackson Browne/Running on Emp...",[https://www.allmusic.com/album/jackson-browne...
4,4,Let Me Roll It,https://www.allmusic.com/song/let-me-roll-it-m...,[Paul McCartney & Wings],[https://www.allmusic.com/artist/paul-mccartne...,"[Linda McCartney, Paul McCartney]",[https://www.allmusic.com/artist/linda-mccartn...,1973,None,None,"[Band on the Run, Pure McCartney, Licorice Piz...",[https://www.allmusic.com/album/band-on-the-ru...


In [4]:
with open('../allmusic/data/genre.jsonl') as f:
    lines = f.readlines()
genre_df = pd.DataFrame([eval(i) for i in lines])
genre_df.sub_genre_url = list(map(lambda x: ['https://www.allmusic.com' + url for url in x] if isinstance(x, list) else None,genre_df.sub_genre_url))

genre_df

,title,url,discription,sub_genre,sub_genre_url
0,Vocal,https://www.allmusic.com/genre/vocal-ma0000011877,Vocal music is a term describi...,NaN,None
1,Religious,https://www.allmusic.com/genre/religious-ma000...,NaN,"[Alternative CCM, CCM, Christian Metal, Christ...",[https://www.allmusic.com/style/alternative-cc...
2,Rap,https://www.allmusic.com/genre/rap-ma0000002816,Rap's germination is sometimes...,"[Afroswing, Cloud Rap, Grime, Instrumental Hip...",[https://www.allmusic.com/style/afroswing-ma00...
3,Reggae,https://www.allmusic.com/genre/reggae-ma000000...,Reggae is a music unique to Ja...,NaN,None
4,Stage & Screen,https://www.allmusic.com/genre/stage-screen-ma...,The incidental music in films ...,"[Cast Recordings, Musical Theater, Musicals, S...",[https://www.allmusic.com/style/cast-recording...
...,...,...,...,...,...
1103,Deep Soul,https://www.allmusic.com/style/deep-soul-ma000...,"Like Southern soul, Deep Soul ...",NaN,None
1104,Go-Go,https://www.allmusic.com/style/go-go-ma0000004428,"Go-Go was a bass-heavy, funky ...",NaN,None
1105,Memphis Soul,https://www.allmusic.com/style/memphis-soul-ma...,Memphis Soul isn't the same th...,NaN,None
1106,Southern Soul,https://www.allmusic.com/style/southern-soul-m...,"Southern Soul is gritty, funky...",NaN,None


## Source: Music Dataset : 1950 to 2019(kaggle)  
https://www.kaggle.com/datasets/saurabhshahane/music-dataset-1950-to-2019

In [5]:
df3 = pd.read_csv('../dataset/tcc_ceds_music.csv')

# df3.id = df3.id.apply(str)
df3.release_date = df3.release_date.apply(str)
df3 = df3[['artist_name', 'track_name', 'release_date', 'genre', 'acousticness','danceability','energy','instrumentalness','loudness','valence']]

In [6]:
df3.columns

Index(['artist_name', 'track_name', 'release_date', 'genre', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'loudness', 'valence'],
      dtype='object')

In [7]:
df3.shape

(28372, 10)

# Source4: 

In [8]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
df4 = pd.read_csv('../dataset/songs_normalize.csv')
df4 = df4[['artist', 'song', 'year', 'genre', 'acousticness','danceability','energy','instrumentalness','loudness','valence']]
df4.loudness = mms.fit_transform(np.array(df4.loudness).reshape(-1, 1))
df4 = df4.rename({'artist': 'artist_name', 'song': 'track_name', 'year': 'release_date'}, axis=1)

In [9]:
df4.columns

Index(['artist_name', 'track_name', 'release_date', 'genre', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'loudness', 'valence'],
      dtype='object')

In [10]:
df_3_4 = pd.concat([df3, df4])
df_3_4 = df_3_4.reset_index()
df_3_4.rename(columns={'index':'id'}, inplace = True)
df_3_4.id = list(map(lambda x: str(x), df_3_4.id))

df_3_4.head()

,id,artist_name,track_name,release_date,genre,acousticness,danceability,energy,instrumentalness,loudness,valence
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,0.997992,0.357739,0.137110,0.901822,0.454119,0.339448
1,1,frankie laine,i believe,1950,pop,0.954819,0.331745,0.263240,0.000002,0.647540,0.325021
2,2,johnnie ray,cry,1950,pop,0.840361,0.456298,0.139112,0.000000,0.585288,0.351814
3,3,pérez prado,patricia,1950,pop,0.083935,0.686992,0.743736,0.199393,0.744404,0.775350
4,4,giorgos papadopoulos,apopse eida oneiro,1950,pop,0.975904,0.291671,0.394375,0.000246,0.646489,0.597073


# 2. Entity Resolution

In [11]:
# !pip install rltk

In [12]:
import rltk
import csv

# You can use this tokenizer in case you need to manipulate some data
tokenizer = rltk.tokenizer.crf_tokenizer.crf_tokenizer.CrfTokenizer()

In [13]:
class SpotifyRecord(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = 'spotify'
        
    @rltk.cached_property
    # id is necessary
    def id(self): 
        return self.raw_object['id']
    
    @rltk.cached_property
    def song_name(self):
        return self.raw_object['track_name']
    
    @rltk.cached_property
    def release_year(self):
        return self.raw_object['release_date']

class AllmusicRecord(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = 'allmusic'
        
    @rltk.cached_property
    def id(self):
        return self.raw_object['id']

    @rltk.cached_property
    def song_name(self):
        return self.raw_object['title']
    
    @rltk.cached_property
    def release_year(self):
        return self.raw_object['release_year']

In [14]:
ds1 = rltk.Dataset(rltk.DataFrameReader(df_3_4), record_class=SpotifyRecord) #Source3&4
ds2 = rltk.Dataset(rltk.DataFrameReader(allmusic_df), record_class=AllmusicRecord) #Source2

In [15]:
# print some entries
print(ds1.generate_dataframe().head(5))
print(ds2.generate_dataframe().head(5))

  id               song_name release_year
0  0  Oops!...I Did It Again         2000
1  1    All The Small Things         1999
2  2                 Breathe         1999
3  3            It's My Life         2000
4  4             Bye Bye Bye         2000
  id                         song_name release_year
0  0  Pulling Mussels (From the Shell)         1980
1  1                     Girls on Film         1983
2  2                          Doodlin'         1955
3  3              Rock Me on the Water         1971
4  4                    Let Me Roll It         1973


## blocking

In [16]:
print('------ block on song_name + release_year ------')
bg = rltk.HashBlockGenerator()
block = bg.generate(bg.block(ds1, function_=lambda r: str.upper(r.song_name[:3]) + str(r.release_year)),
                    bg.block(ds2, function_=lambda r: str.upper(r.song_name[:3]) + str(r.release_year))) #property_='song_name'
pairs = rltk.get_record_pairs(ds1, ds2, block=block)
num = 0
for r1, r2 in pairs:
    num += 1
    if num <= 5:
        print(r1.id, r1.song_name, '\t', r2.id, r2.song_name)
print(f'There are {num} pairs with blocking.')

------ block on song_name + release_year ------
0 Oops!...I Did It Again 	 6746 Oops!...I Did It Again
1 All The Small Things 	 34456 All Star
11040 all night long 	 34456 All Star
11011 all the way 	 34456 All Star
22144 all day all night 	 34456 All Star
There are 73896 pairs with blocking.


## define similarity measures

In [17]:
def jw_similarity(r1, r2):
    s1 = str.upper(r1.song_name)
    s2 = str.upper(r2.song_name)
    return rltk.jaro_winkler_similarity(s1, s2)

def l_distance(r1, r2):
    for n1, n2 in zip(sorted(str.upper(r1.song_name)), sorted(str.upper(r2.song_name))):
        if rltk.levenshtein_distance(n1, n2) > min(len(n1), len(n2)) / 3:
            return 0
    return 1
    
def name_string_similarity(r1, r2):
    s1 = str.upper(r1.song_name)
    s2 = str.upper(r2.song_name)
    if s1 == s2:
        return 1
    return 0

# threshold value to determine if we are confident the record match
MY_TRESH = 0.7

# entity linkage scoring function
def rule_based_method(r1, r2):
    score_1 = jw_similarity(r1, r2)
    score_2 = l_distance(r1, r2)
    score_3 = name_string_similarity(r1, r2)
    
    total = 0.8 * score_1 + 0.1 * score_2 + 0.1 * score_3
    
    # return two values: boolean if they match or not, float to determine confidence
    return total > MY_TRESH, total

## entity linking

In [20]:
i = 0
matching_pairs = []
for r1, r2 in rltk.get_record_pairs(ds1, ds2, block=block):
    result, confidence = rule_based_method(r1, r2)
    if result:
        i += 1
        matching_pairs.append((r1.id, r2.id))
print(f'There are {i} matching pairs.')

with open('../kg_data/er_result.csv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    header = ['spotify.id', 'allmusic.id']
    writer.writerow(header)
    for row in matching_pairs:
        writer.writerow(row)

There are 4920 matching pairs.


# 3. Define ontology and Generate RDF triples

In [21]:
from rdflib import Graph, URIRef, Literal, XSD, Namespace, RDF

In [22]:
df = pd.read_csv('../kg_data/er_result.csv', dtype=object)
df = df.drop_duplicates(subset=['spotify.id']).reset_index(drop=True)
df

,spotify.id,allmusic.id
0,0,6746
1,6,2048
2,25980,23087
3,11063,26450
4,15447,20158
...,...,...
2009,27294,9637
2010,27344,11183
2011,27348,20265
2012,27449,9636


In [24]:
MYNS = Namespace('http://inf558.org/mynamespace#')
FOAF = Namespace('http://xmlns.com/foaf/0.1/')
SCHEMA = Namespace('http://schema.org/')

kg = Graph()
kg.bind('my_ns', MYNS)
kg.bind('foaf', FOAF)
kg.bind('schema', SCHEMA)

genre_set = set()
artist_set = set()

for i in range(len(df)):    # len(df)
    spotify_id = df['spotify.id'][i]
    allmusic_id = df['allmusic.id'][i]
    
    song_uri = URIRef(allmusic_df[allmusic_df.id == allmusic_id]['url'].array[0])
    kg.add((song_uri, RDF.type, MYNS['song']))
    
    song_name = allmusic_df[allmusic_df.id == allmusic_id]['title'].array[0]
    kg.add((song_uri, FOAF['name'], Literal(song_name)))
    
    song_genre_list = allmusic_df[allmusic_df.id == allmusic_id]['genres'].array[0]
    if isinstance(song_genre_list, list):
        for genre in song_genre_list:
            genre_set.add(genre)
            genre_uri = URIRef(genre_df[genre_df.title == genre]['url'].array[0])
            kg.add((song_uri, MYNS.belongs_to, URIRef(genre_uri)))
            
    song_style_list = allmusic_df[allmusic_df.id == allmusic_id]['styles'].array[0]
    if isinstance(song_style_list, list):
        for style in song_style_list:
            if not (genre_df[genre_df.title == style].empty):
                genre_set.add(style)
                style_uri = URIRef(genre_df[genre_df.title == style]['url'].array[0])
                kg.add((song_uri, MYNS.belongs_to, URIRef(style_uri)))
                
    composer_uri_list = allmusic_df[allmusic_df.id == allmusic_id]['composer_url'].array[0]
    if isinstance(composer_uri_list, list):
        for composer_uri in composer_uri_list:
            artist_set.add(composer_uri)
            kg.add((song_uri, MYNS.composed_by, URIRef(composer_uri)))
            
    performer_uri_list = allmusic_df[allmusic_df.id == allmusic_id]['performer_url'].array[0]
    if isinstance(performer_uri_list, list):
        for performer_uri in performer_uri_list:
            artist_set.add(performer_uri)
            kg.add((song_uri, MYNS.performed_by, URIRef(performer_uri)))
    
    release_year = allmusic_df[allmusic_df.id == allmusic_id]['release_year'].array[0]
    kg.add((song_uri, MYNS.release_year, Literal(release_year)))
    
    acousticness = df_3_4[df_3_4.id == spotify_id]['acousticness'].array[0]
    kg.add((song_uri, MYNS.acousticness, Literal(acousticness)))
    
    danceability = df_3_4[df_3_4.id == spotify_id]['danceability'].array[0]
    kg.add((song_uri, MYNS.danceability, Literal(danceability)))
    
    energy = df_3_4[df_3_4.id == spotify_id]['energy'].array[0]
    kg.add((song_uri, MYNS.energy, Literal(energy)))
    
    instrumentalness = df_3_4[df_3_4.id == spotify_id]['instrumentalness'].array[0]
    kg.add((song_uri, MYNS.instrumentalness, Literal(instrumentalness)))
    
    loudness = df_3_4[df_3_4.id == spotify_id]['loudness'].array[0]
    kg.add((song_uri, MYNS.loudness, Literal(loudness)))
    
    valence = df_3_4[df_3_4.id == spotify_id]['valence'].array[0]
    kg.add((song_uri, MYNS.valence, Literal(valence)))

for i in range(len(genre_df)):
    
    genre_uri = URIRef(genre_df['url'][i])
    kg.add((genre_uri, RDF.type, MYNS['genre']))
    
    genre_title = genre_df['title'][i]
    kg.add((genre_uri, FOAF['name'], Literal(genre_title)))

    genre_discription = genre_df['discription'][i]
    kg.add((genre_uri, MYNS.discription, Literal(genre_discription)))
    
    sub_genre_uri_list = genre_df['sub_genre_url'][i]
    if isinstance(sub_genre_uri_list, list):
        for sub_genre_uri in sub_genre_uri_list:
            sub_genre_uri = URIRef(sub_genre_uri)
            kg.add((sub_genre_uri, MYNS.subgenre_of, genre_uri))
            
for i in range(len(artist_df)):
    
    group_members_uri_list = artist_df['group_members_url'][i]
    
    if isinstance(group_members_uri_list, list):
        this_uri = URIRef(artist_df['url'][i])
        kg.add((this_uri, RDF.type, MYNS['group']))
        
        for member_url in group_members_uri_list:
            member_uri = URIRef(member_url)
            kg.add((member_uri, MYNS.member_of, this_uri))
    else:
        this_uri = URIRef(artist_df['url'][i])
        kg.add((this_uri, RDF.type, MYNS['artist']))
    
    name = artist_df['name'][i]
    kg.add((this_uri, FOAF['name'], Literal(name)))

    discription = artist_df['biography'][i]
    if discription:
        kg.add((this_uri, MYNS.discription, Literal(discription)))
    
    active_years = artist_df['active_years'][i]
    if active_years:
        kg.add((this_uri, MYNS.active_years, Literal(active_years)))
    
    born_in = artist_df['born_in'][i]
    if born_in:
        kg.add((this_uri, MYNS.born_in, Literal(born_in)))
    
    aliases = artist_df['aliases'][i]
    if aliases:
        kg.add((this_uri, MYNS.aliases, Literal(aliases)))
           
kg.serialize('../kg_data/music_kg.ttl', format="turtle")

<Graph identifier=N4f1e47531f4140cd83c9ba5832812af4 (<class 'rdflib.graph.Graph'>)>

In [25]:
with open('../kg_data/music_kg.csv', 'w', newline='', encoding='utf8') as f:
    writer = csv.writer(f, delimiter=',')
    header = ['s', 'p', 'o']
    writer.writerow(header)
    for s, p, o in kg:
        writer.writerow((str(s),p[p.find('#')+1:],o))